In [172]:
%pip install -q google-genai google-adk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [173]:
import os
from dotenv import load_dotenv

load_dotenv()  # This loads variables from .env into the environment

# Set your Google Gemini API key here
api_key = os.getenv("GOOGLE_API_KEY")

In [174]:
# Import necessary genai modules
from google import genai

client = genai.Client()

from google.adk.agents import Agent  # Import Agent class from google.adk.agents
from google.adk.runners import Runner  # Import Runner class from google.adk.runners
from google.adk.sessions import InMemorySessionService  # Import in-memory session service for agent sessions
from google.adk.tools import google_search  # Import Google search tool for agent use
from google.genai import types  # Import types for creating content and parts
from datetime import date  # Import date class for working with dates
import warnings  # Import warnings module to handle warning messages

warnings.filterwarnings("ignore")  # Ignore all warning messages

In [175]:
# Create an in-memory session service
session_service = InMemorySessionService()

# Helper function that sends a message to an agent via Runner and returns the final response
async def call_agent(agent: Agent, message_text: str) -> str:
    # Create a new session for the agent
    session = await session_service.create_session(app_name=agent.name, user_id="user1")
    # Create a Runner instance with the agent and session service
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Create content with the user's message
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # iterate over the events returned by the runner
    async for event in runner.run_async(user_id="user1", session_id=session.id, new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [176]:
FAST_MODEL = "gemini-2.0-flash"
STRONG_MODEL = "gemini-2.5-pro-preview-tts"

In [177]:
import textwrap  # Import textwrap for formatting text output
from IPython.display import display, Markdown  # Import display and Markdown for formatted output in notebooks

# Helper function to display formatted text in Markdown in Colab
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [178]:
##########################################
# --- Agent 1: News Searcher --- #
##########################################
async def agent_search(topic, today):
    print(f"Searching for the last 10 games of: {topic} with date: {today}")
    # Create an agent that will search for news on Google
    if not topic or not today:
        raise ValueError("Both 'topic' and 'today' must be provided.")
    searcher = Agent(
        name="agent_search",
        model=FAST_MODEL,
        instruction="""
        You are a soccer research assistant. Your task is to use the google_search tool
        to retrieve the result of the last 10 official games of the soccer team informed below.
        You should provide a table with the following columns: Date, Opponent, Result, and a column with L or W (Loss or Win).
        """,
        description="Agent that searches for the results of soccer games on Google",
        tools=[google_search]
    )

    entrance_agent_search = f"Soccer team: {topic}\nToday date: {today}"

    news = await call_agent(searcher, entrance_agent_search)
    return news

In [179]:
################################################
# --- Agent 2: Post Percentage success --- #
################################################
async def agent_perc_success(topic, searched_releases):
    planner = Agent(
        name="agent_perc_success",
        model=FAST_MODEL,
        # Insert the agent_perc_success Agent instructions #################################################
        instruction="""
        You are a statistics expert. Your task is to analyze the results of the last 10 games
        of the soccer team informed below and calculate the percentage of success.
        The percentage of success is defined by the sum of number of wins multiplied by 3 
        and the sum of number of draws multiplied by 1,
        divided by the total number of games played multiplied by 3.
        You should return the percentage of success in the format: "The percentage of success is X%".
        """,
        description="Agent that provide the percentage of success of the soccer team informed",
        tools=[google_search]
    )

    planner_agent_input = f"Topic:{topic}\nPercentage of success: {searched_releases}"
    # Runs the agent
    post_plan = await call_agent(planner, planner_agent_input)
    return post_plan

In [180]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Starting the agent to get information of the last 10 games of soccer team 🚀")

# --- Get the User Topic ---
topico = input("❓ Please enter the name of the soccer team: ")

# Insert agent system logic ################################################
if not topico:
    print("You forgot to enter the name of the soccer team!")
else:
    print(f"Great! Let's create the post about news in {topico}")

    search_games = await agent_search(topico, data_de_hoje)
    print("\n--- Result of Agent 1 (Searcher of Games) ---\n")
    display(to_markdown(search_games))
    print("--------------------------------------------------------------")

    post_results_metrics = await agent_perc_success(topico, search_games)
    print("\n--- 📝 Result of Agent 2 (Metrics for the results) ---\n")
    display(to_markdown(post_results_metrics))


🚀 Starting the agent to get information of the last 10 games of soccer team 🚀
Great! Let's create the post about news in Flamengo
Searching for the last 10 games of: Flamengo with date: 09/06/2025

--- Result of Agent 1 (Searcher of Games) ---



> Okay, I will search for the results of the last 10 official games of Flamengo and present them in a table with the columns: Date, Opponent, Result, and L/W (Loss/Win).
> 
> 
> Based on the search results, here's a table of Flamengo's last 10 official game results:
> 
> | Date       | Opponent          | Result | L/W |
> |------------|-------------------|--------|-----|
> | 2025-06-01 | Fortaleza         | 5 - 0  | W   |
> | 2025-05-28 | Deportivo Táchira  | 1 - 0  | W   |
> | 2025-05-25 | Palmeiras         | 0 - 2  | W   |
> | 2025-05-21 | Botafogo-PB       | 4 - 2  | W   |
> | 2025-05-18 | Botafogo RJ       | 0 - 0  | D   |
> | 2025-05-15 | LDU Quito         | 2 - 0  | W   |
> | 2025-05-10 | Bahia             | 1 - 0  | W   |
> | 2025-05-07 | Central Córdoba   | 1 - 1  | D   |
> | 2025-05-04 | Cruzeiro          | 2 - 1  | L   |
> | 2025-05-01 | Botafogo-PB       | 0 - 1  | W   |
> 
> **Note:**
> *   D stands for Draw.
> 


--------------------------------------------------------------

--- 📝 Result of Agent 2 (Metrics for the results) ---



> Based on the provided results, here's the calculation of Flamengo's percentage of success:
> 
> *   **Wins:** 7
> *   **Draws:** 2
> *   **Losses:** 1
> *   **Total Games:** 10
> 
> Success Points: (7 Wins \* 3 points/win) + (2 Draws \* 1 point/draw) = 21 + 2 = 23
> 
> Total Possible Points: 10 Games \* 3 points/game = 30
> 
> Percentage of Success: (23 / 30) \* 100 = 76.67%
> 
> The percentage of success is 76.67%.
> 
